# Proxy pattern
- The proxy design pattern gets its name from the proxy (also known as surrogate) object used to perform an important action before accessing the actual object. There are four well-known types of proxy. They are as follows:

In [1]:
class LazyProperty:
    def __init__(self, method):
        self.method = method
        self.method_name = method.__name__
        print(f"function overriden: {self.method}")
        print(f"function's name: {self.method_name}")

    def __get__(self, obj, cls):
        if not obj:
            print("obj is None")
            return None

        print(f"The object is: {obj}")
        value = self.method(obj)
        print(f"value {value}")
        setattr(obj, self.method_name, value)
        return value

In [2]:
class Test:
    def __init__(self):
        self.x = "foo"
        self.y = "bar"
        self._resource = None

    @LazyProperty
    def resource(self):
        print("initializing self._resource...")
        print(f"... which is: {self._resource}")
        self._resource = tuple(range(5))  # expensive
        return self._resource

function overriden: <function Test.resource at 0x701b111019e0>
function's name: resource


In [3]:
def main():
    t = Test()
    print(t.x)
    print(t.y)
    # do more work...
    print(t.resource)
    print(t.resource)
    print(t.resource)

In [4]:
main()

foo
bar
The object is: <__main__.Test object at 0x701b1110c290>
initializing self._resource...
... which is: None
value (0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)


<hr>

In [5]:
class SensitiveInfo:
    def __init__(self):
        self.users = ["nick", "tom", "ben", "mike"]

    def read(self):
        nb = len(self.users)
        print(f"There are {nb} users: {' '.join(self.users)}")

    def add(self, user):
        self.users.append(user)
        print(f"Added user {user}")

In [6]:
class Info:
    """protection proxy to SensitiveInfo"""

    def __init__(self):
        self.protected = SensitiveInfo()
        self.secret = "0xdeadbeef"

    def read(self):
        self.protected.read()

    def add(self, user):
        sec = input("what is the secret? ")
        if sec == self.secret:
            self.protected.add(user)
        else:
            print("That's wrong!")

In [7]:
def main():
    info = Info()
    while True:
        print("1. read list |==| 2. add user |==| 3. quit")
        key = input("choose option: ")
        if key == "1":
            info.read()
        elif key == "2":
            name = input("choose username: ")
            info.add(name)
        elif key == "3":
            break
        else:
            print(f"unknown option: {key}")

In [8]:
main()

1. read list |==| 2. add user |==| 3. quit
unknown option: 
1. read list |==| 2. add user |==| 3. quit
There are 4 users: nick tom ben mike
1. read list |==| 2. add user |==| 3. quit
That's wrong!
1. read list |==| 2. add user |==| 3. quit
unknown option: 0xdeadbeef
1. read list |==| 2. add user |==| 3. quit
There are 4 users: nick tom ben mike
1. read list |==| 2. add user |==| 3. quit
There are 4 users: nick tom ben mike
1. read list |==| 2. add user |==| 3. quit
That's wrong!
1. read list |==| 2. add user |==| 3. quit
unknown option: 0xdeadbeef
1. read list |==| 2. add user |==| 3. quit


<hr>

In [9]:
from abc import ABC, abstractmethod

In [10]:
class RemoteServiceInterface(ABC):
    @abstractmethod
    def read_file(self, file_name):
        pass

    @abstractmethod
    def write_file(self, file_name, contents):
        pass

    @abstractmethod
    def delete_file(self, file_name):
        pass

In [11]:
class RemoteService(RemoteServiceInterface):
    def read_file(self, file_name):
        # Implementation for reading a file from the server
        return "Reading file from remote server"

    def write_file(self, file_name, contents):
        # Implementation for writing to a file on the server
        return "Writing to file on remote server"

    def delete_file(self, file_name):
        # Implementation for deleting a file from the server
        return "Deleting file from remote server"

In [12]:
class ProxyService(RemoteServiceInterface):
    def __init__(self):
        self.remote_service = RemoteService()

    def read_file(self, file_name):
        print("Proxy: Forwarding read request to RemoteService")
        return self.remote_service.read_file(file_name)

    def write_file(self, file_name, contents):
        print("Proxy: Forwarding write request to RemoteService")
        return self.remote_service.write_file(file_name, contents)

    def delete_file(self, file_name):
        print("Proxy: Forwarding delete request to RemoteService")
        return self.remote_service.delete_file(file_name)

In [13]:
proxy = ProxyService()
print(proxy.read_file("example.txt"))

Proxy: Forwarding read request to RemoteService
Reading file from remote server


<hr>

In [14]:
from typing import Protocol

In [15]:
class DBConnectionInterface(Protocol):
    def exec_query(self, query): ...

In [16]:
class DBConnection:
    def __init__(self):
        print("DB connection created")

    def exec_query(self, query):
        return f"Executing query: {query}"

    def close(self):
        print("DB connection closed")

In [18]:
class SmartProxy:
    def __init__(self):
        self.cnx = None
        self.ref_count = 0

    def access_resource(self):
        if self.cnx is None:
            self.cnx = DBConnection()
        self.ref_count += 1
        print(f"DB connection now has {self.ref_count} references.")

    def exec_query(self, query):
        if self.cnx is None:
            # Ensure the connection is created
            # if not already
            self.access_resource()

        result = self.cnx.exec_query(query)
        print(result)

        # Decrement reference count after
        # executing query
        self.release_resource()

        return result

    def release_resource(self):
        if self.ref_count > 0:
            self.ref_count -= 1
            print("Reference released...")
            print(f"{self.ref_count} remaining refs.")

        if self.ref_count == 0 and self.cnx is not None:
            self.cnx.close()
            self.cnx = None

In [19]:
proxy = SmartProxy()
proxy.exec_query("SELECT * FROM users")
proxy.exec_query("UPDATE users SET name = 'John Doe' WHERE id = 1")

DB connection created
DB connection now has 1 references.
Executing query: SELECT * FROM users
Reference released...
0 remaining refs.
DB connection closed
DB connection created
DB connection now has 1 references.
Executing query: UPDATE users SET name = 'John Doe' WHERE id = 1
Reference released...
0 remaining refs.
DB connection closed


"Executing query: UPDATE users SET name = 'John Doe' WHERE id = 1"